In [99]:
import pypylon.pylon as py
import numpy as np
import cv2
import time

# this sample has been tested with a Basler acA1920-155uc
# type 'q' or 'ESC' in the window to close it

# the camera is configured to run at high framerate with only two lines hight
# the acquired rows are concatenated as a virtual frame and this frame is displayed



tlf = py.TlFactory.GetInstance()
cameras = tlf.EnumerateDevices()
for c in cameras:
    print(c.GetModelName(), c.GetSerialNumber())
    
camera = py.InstantCamera(tlf.CreateDevice(cameras[0]))
camera.Open()

camera.PixelFormat.Value = "Mono8"
camera.ExposureAuto.Value = "Off"
camera.Gain.Value = 0

max_exposure = 20000
min_exposure = 100
step_exposure = 10
camera.ExposureTime.Value=100

camera.AcquisitionFrameRateEnable.Value = False
camera.AcquisitionFrameRate.Value = 100

camera.StartGrabbing(py.GrabStrategy_LatestImageOnly)

converter = py.ImageFormatConverter()
converter.OutputPixelFormat = py.PixelType_Mono8
converter.OutputBitAlignment = py.OutputBitAlignment_MsbAligned

desired_fps = 160
frame_duration = 1.0 / desired_fps

try:
    while camera.IsGrabbing():
        loop_start = time.time()
        grabResult = camera.RetrieveResult(5000, py.TimeoutHandling_ThrowException)
        if grabResult.GrabSucceeded():
            image = converter.Convert(grabResult)
            frame = image.GetArray()
            current_fps = camera.ResultingFrameRate.Value
            fps_msg = f"FPS: {current_fps:.2f}"
            fontScale = 1.5
            thickness = 1
            (fpsWidth, fpsHeight), _ = cv2.getTextSize(fps_msg, cv2.FONT_HERSHEY_SIMPLEX, fontScale, thickness)
            rectFPS, posFPS = textBackground(fps_msg, fontScale, thickness, (5, 5))
            cv2.rectangle(frame, rectFPS[0], rectFPS[1], color=(255, 255, 255), thickness=-1)
            cv2.putText(frame, fps_msg, (5, textHeight+5), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0,0,0), thickness, lineType=cv2.LINE_AA)

            exposure_msg = f'Exp: {camera.ExposureTime.Value:.2f}' 
            (exposureWidth, exposureHeight), _ = cv2.getTextSize(exposure_msg, cv2.FONT_HERSHEY_SIMPLEX, fontScale, thickness)
            rectExposure, posExposure = textBackground(exposure_msg, fontScale, thickness, (5, fpsHeight+25))
            cv2.rectangle(frame, rectExposure[0], rectExposure[1], color=(255, 255, 255), thickness=-1)
            cv2.putText(frame, exposure_msg, (5, fpsHeight+exposureHeight+25), cv2.FONT_HERSHEY_SIMPLEX, fontScale, (0,0,0), thickness, lineType=cv2.LINE_AA)
            
            frame = cv2.resize(frame, (0, 0), fx = 0.5, fy = 0.5)

            exposure = camera.ExposureTime.Value
            if current_fps < desired_fps and exposure > min_exposure:
                camera.ExposureTime.Value-=step_exposure
            elif current_fps > desired_fps and exposure < max_exposure:
                camera.ExposureTime.Value+=step_exposure
            
            cv2.imshow("Camera Feed", frame)
        
        grabResult.Release()


        if cv2.waitKey(1) & 0xFF in (ord('q'), 27):
            break
        
        elapsed = time.time() - loop_start
        time_to_wait = frame_duration - elapsed
        if time_to_wait > 0:
            time.sleep(time_to_wait)
finally:
    camera.StopGrabbing()
    camera.Close()
    cv2.destroyAllWindows()

acA1300-200um 23769231


In [83]:
camera.Close()

In [66]:
def textBackground(text, fontScale, thickness, pos):
    (textWidth, textHeight), baseline = cv2.getTextSize(str(text), cv2.FONT_HERSHEY_SIMPLEX, fontScale, thickness)
    rect = [(pos[0], pos[1]), (pos[0] + textWidth, pos[1] + textHeight+baseline)]
    textPos = (pos[0], pos[1] + textHeight + baseline // 2)
    
    return (rect, textPos)

In [95]:
tlf = py.TlFactory.GetInstance()
cameras = tlf.EnumerateDevices()
for c in cameras:
    print(c.GetModelName(), c.GetSerialNumber())

acA1300-200um 23769231


In [96]:
cameras[0]

<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x000001AB1585AB80> >

In [98]:
tlf = py.TlFactory.GetInstance()
cameras = tlf.EnumerateDevices()
for c in cameras:
    print(c)

<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x000001AB7F1BE970> >
